# Unziping the files

In [ ]:
import zipfile
import os

def extract(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:

      zip_ref.extractall(extracted_dir)

      for filename in os.listdir(extracted_dir):
          file_path = os.path.join(extracted_dir, filename)
          if os.path.isfile(file_path):
              with open(file_path, 'r', encoding='utf-8') as file:

                  content = file.read()

# extracting :
easy_ham = 'easy_ham.zip'
hard_ham = 'hard_ham.zip'
spam = 'spam_2.zip'

extracted_dir = 'data/'

extract(easy_ham)
extract(hard_ham)
extract(spam)

# cleaning data

In [10]:
import pandas as pd
import os

# get the data
data = []
for folder in ['data/easy_ham', 'data/spam_2' , 'data/hard_ham']:
    
    lable = 0
    for file in os.listdir(folder):
        with open(f'{folder}/{file}', 'r', errors='replace') as f:
            if folder == 'spam_2':
                label = 1
            else:
                label = 0
            email_str = f.read()
            data.append([email_str, label])

# create a dataframe
df = pd.DataFrame(data)
df.columns = ['email', 'label']

# save the dataframe to a csv file
df.to_csv('emails.csv', index=False)
print('done')


done


In [11]:
# read the csv file
df = pd.read_csv('emails.csv')
print(df)

                                                  email  label
0     From exmh-workers-admin@redhat.com  Thu Aug 22...      0
1     From Steve_Burt@cursor-system.com  Thu Aug 22 ...      0
2     From timc@2ubh.com  Thu Aug 22 13:52:59 2002\n...      0
3     From irregulars-admin@tb.tf  Thu Aug 22 14:23:...      0
4     From exmh-users-admin@redhat.com  Thu Aug 22 1...      0
...                                                 ...    ...
4193  Return-Path: ler@lerami.lerctr.org\nDelivery-D...      0
4194  Return-Path: ler@lerami.lerctr.org\nDelivery-D...      0
4195  From bounce-ora_webprog-1083425@newsletter.ore...      0
4196  Return-Path: linux-announce-recipients-owner-m...      0
4197  Return-Path: <test-admin@lists.sourceforge.net...      0

[4198 rows x 2 columns]


# Data preparation

In [12]:
# extract the email body from the email string

def extract_email_body(email_str):
    # check if the email string contains at least two new lines
    if '\n\n' not in email_str:
        return email_str

    # the email body is between the first two new lines
    body = email_str.split('\n\n', 1)[1]

    # remove the footer 
    while body[-1] == '\n':
        body = body[:-1]

    body = body.split('\n\n')[:-1]
    body = '\n\n'.join(body)

    return body
        
    


test = extract_email_body(df['email'][598])
print(test)
print("-------------------------------------")
#print(df[0][598])

On Mon, 9 Sep 2002, Ned Jackson Lovely wrote:

> In '87 a guy named Gregory Robertson noticed a fellow parachutist Debbie 
> Williams had been knocked unconscious. He shifted so that he was head down,
> hit about 200 mi/h, and caught up with her and pulled her chute with 10 seconds
> to spare.
-------------------------------------


In [13]:
df['email'] = df['email'].apply(extract_email_body)
        
print(df['email'][598])

On Mon, 9 Sep 2002, Ned Jackson Lovely wrote:

> In '87 a guy named Gregory Robertson noticed a fellow parachutist Debbie 
> Williams had been knocked unconscious. He shifted so that he was head down,
> hit about 200 mi/h, and caught up with her and pulled her chute with 10 seconds
> to spare.


In [22]:
import re 

def convert_url(words):
    for i, word in enumerate(words):
        url = re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|" \
                       "[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", word)
        
        if url:
            words[i] = 'URL'

    return words

def convert_num(words):
    words = list(words)  # convert string to list of characters
    for i, word in enumerate(words):
        if word.isdigit():
            words[i] = 'NUM '
    return ''.join(words)  # convert list back to string

def clean_text(text):
  text = re.sub(r'[^\w\s]', '', text) #to remove ponctuation
  text = re.sub(r'\d+', '', text) #to remove spacing
  text = text.lower()

  return text



In [23]:
df['email'] = df['email'].apply(convert_url)

df['email'] = df['email'].apply(convert_num)

df['email'] = df['email'].apply(clean_text)

print(df['email'][598])

on mon num sep numnumnumnum ned jackson lovely wrote

 in numnum a guy named gregory robertson noticed a fellow parachutist debbie 
 williams had been knocked unconscious he shifted so that he was head down
 hit about numnumnum mih and caught up with her and pulled her chute with numnum seconds
 to spare


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import numpy as np

# Splitting data

In [26]:
df

,email,label
0,date wed numnum aug numnumnumnum nu...,0
1,martin a posted\ntassos papadopoulos the greek...,0
2,man threatens explosion in moscow \n\nthursday...,0
3,klez the virus that wont die\n \nalready the m...,0
4,on wed aug numnum numnumnumnum at numnumnumnum...,0
...,...,...
4193,greetings from geocachingcom \n\n\nrecent cach...,0
4194,i am trying to secure three of four virtual ho...,0
4195,filled with useful examples and the depth clar...,0
4196,linuxannounce digest numnumnum volume num ...,0


In [27]:
df.shape

(4198, 2)

In [31]:
# splitting the emails and the labels
X = df['email']
y = df['label']

# splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(3358,) (840,)
(3358,) (840,)


# Feature extraction

In [33]:
#transform the emails to numerical data
vectorizer = TfidfVectorizer(min_df=1 , stop_words='english' , lowercase=True)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(X_train_vec.shape, X_test_vec.shape)

(3358, 84743) (840, 84743)


In [34]:
# convert the labels to ints

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [38]:
print(X_train)

964     when brickbats or fox exposes the folly of gov...
239     hardware acceleration for ssl makes sense sinc...
2387    url httpwwwnewsisfreecomclicknumnumnumnumnumnu...
990     on fri numnum aug numnumnumnum robert harley w...
108     on fri numnum aug numnumnumnum tom wrote\n\n f...
                              ...                        
3444    barrister adewale coker chambers\nlegal practi...
466     i used async io on system v in the numnum numn...
3092                                                     
3772    we are offering you quality marketing lists wh...
860      g  gregory alan bolcer gbolcerendeavorscom wr...
Name: email, Length: 3358, dtype: object


In [37]:
print(X_train_vec)

  (0, 70950)	0.07682256555657738
  (0, 53153)	0.1286741397315909
  (0, 78739)	0.12047917664135062
  (0, 58338)	0.11293910383616924
  (0, 76250)	0.09823943260143536
  (0, 58582)	0.06538914770810232
  (0, 82560)	0.090699359796254
  (0, 81241)	0.07194568977238554
  (0, 12305)	0.07179068775287223
  (0, 22740)	0.06613052113639543
  (0, 73620)	0.0912086909627938
  (0, 26597)	0.1286741397315909
  (0, 29713)	0.10783028303500052
  (0, 29158)	0.11104955956713962
  (0, 43633)	0.26918035258514506
  (0, 28565)	0.109359304621393
  (0, 45852)	0.1286741397315909
  (0, 59844)	0.06146125068614039
  (0, 73291)	0.06409709622383931
  (0, 64978)	0.1286741397315909
  (0, 42808)	0.1286741397315909
  (0, 40374)	0.11755426771163326
  (0, 23100)	0.2258782076723385
  (0, 70860)	0.07599968856152012
  (0, 46711)	0.07341253072194577
  :	:
  (3357, 16128)	0.06774345321244579
  (3357, 10185)	0.16393676630073095
  (3357, 70359)	0.05014330562163666
  (3357, 49009)	0.05552798717988132
  (3357, 82707)	0.05942893516608282


# Training the model

In [41]:
model = LogisticRegression()

model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0